# Jack The Walker - Colab Training with Google Drive

**SETUP:**
1. Runtime → Change runtime type → **T4 GPU**
2. Upload `JackBrain.py` and `SOTATrainer.py` to Google Drive folder: `JackTheWalker/`
3. Run cells below in order

**BENEFITS:**
- ✅ No manual download needed
- ✅ Checkpoints auto-save to Drive
- ✅ Resume anytime from any device
- ✅ Access datasets from Drive

In [ ]:
# Cell 1: Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')
print("✅ Google Drive mounted!")

# Create project folder if doesn't exist
os.makedirs('/content/drive/MyDrive/JackTheWalker', exist_ok=True)
os.makedirs('/content/drive/MyDrive/JackTheWalker/checkpoints', exist_ok=True)
os.makedirs('/content/drive/MyDrive/JackTheWalker/datasets', exist_ok=True)

print("\n📁 Folders created in Google Drive:")
print("   - JackTheWalker/")
print("   - JackTheWalker/checkpoints/")
print("   - JackTheWalker/datasets/")
print("\n⚠️ Now upload JackBrain.py and SOTATrainer.py to JackTheWalker/ folder in Google Drive")

In [ ]:
# Cell 2: Install dependencies
!pip install -q gymnasium[mujoco]
print("✅ Dependencies installed!")

In [ ]:
# Cell 3: Verify GPU
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("\n✅ GPU ready! Training will be 10-20x faster than CPU!")
else:
    print("\n⚠️ No GPU! Go to Runtime → Change runtime type → T4 GPU")

In [ ]:
# Cell 4: Change to project directory
import os

os.chdir('/content/drive/MyDrive/JackTheWalker')
print(f"Working directory: {os.getcwd()}")

# Check files exist
import os
files = os.listdir('.')
print(f"\nFiles in directory: {files}")

if 'JackBrain.py' not in files or 'SOTATrainer.py' not in files:
    print("\n⚠️ ERROR: JackBrain.py or SOTATrainer.py not found!")
    print("Please upload them to Google Drive/JackTheWalker/ folder")
else:
    print("\n✅ Files found! Ready to train!")

In [ ]:
# Cell 5: PHASE 1 - RL Training (PPO)
# This trains basic locomotion
# Runs for ~12 hours then auto-disconnects (Colab limit)
# Checkpoint saves to Drive automatically every 10 epochs!

!python -u SOTATrainer.py --no-render --epochs 1000

print("\n✅ Training session complete!")
print("Checkpoint saved to: /content/drive/MyDrive/JackTheWalker/checkpoints/latest.pt")
print("\nTo continue training:")
print("1. Just re-run this cell - it will auto-resume from latest.pt")
print("2. Or start Phase 2 (datasets) once locomotion is good")

---

## PHASE 2: Dataset Training (Behavior Cloning)

**Run this AFTER Phase 1 completes (when robot can walk)**

### Setup:
1. Download CMU MoCap dataset (3 tar files)
2. Upload to: `Google Drive/JackTheWalker/datasets/cmu_mocap/`
3. Run Cell 6 below

In [ ]:
# Cell 6: Check dataset availability
import os

dataset_path = '/content/drive/MyDrive/JackTheWalker/datasets/cmu_mocap'

if os.path.exists(dataset_path):
    files = os.listdir(dataset_path)
    print(f"✅ CMU MoCap dataset found!")
    print(f"Files: {len(files)} files")
    print(f"\nReady for Phase 2 training!")
else:
    print("⚠️ Dataset not found!")
    print(f"\nPlease upload CMU MoCap dataset to:")
    print(f"Google Drive/JackTheWalker/datasets/cmu_mocap/")
    print(f"\nHow to get it:")
    print(f"1. Download from CMU Graphics Lab")
    print(f"2. Extract all 3 tar files")
    print(f"3. Upload all .bvh files to the folder above")

In [ ]:
# Cell 7: PHASE 2 - Behavior Cloning on CMU MoCap
# This adds natural human movement to the walking robot
# Loads checkpoints/latest.pt and improves it with demonstrations

!python TrainingJack.py \
    --dataset cmu_mocap \
    --checkpoint-in checkpoints/latest.pt \
    --checkpoint-out checkpoints/natural_movement.pt

print("\n✅ Phase 2 complete!")
print("Saved to: checkpoints/natural_movement.pt")
print("\nRobot now has:")
print("  ✅ Basic locomotion (Phase 1)")
print("  ✅ Natural human-like movement (Phase 2)")

In [ ]:
# Cell 8: Download final checkpoint (optional)
# Checkpoints are already in Google Drive, but you can download to local PC

from google.colab import files

print("Downloading checkpoints...")
files.download('/content/drive/MyDrive/JackTheWalker/checkpoints/latest.pt')

try:
    files.download('/content/drive/MyDrive/JackTheWalker/checkpoints/best.pt')
except:
    print("No best.pt (that's okay)")

try:
    files.download('/content/drive/MyDrive/JackTheWalker/checkpoints/natural_movement.pt')
except:
    print("No natural_movement.pt yet (run Phase 2 first)")

print("\n✅ Downloads complete!")

---

## How It Works:

### File Structure in Google Drive:
```
Google Drive/
└── MyDrive/
    └── JackTheWalker/
        ├── JackBrain.py              ← Upload this
        ├── SOTATrainer.py            ← Upload this
        ├── TrainingJack.py           ← Upload for Phase 2
        ├── checkpoints/
        │   ├── latest.pt             ← Auto-saves here!
        │   ├── best.pt               ← Auto-saves here!
        │   └── natural_movement.pt   ← Phase 2 output
        └── datasets/
            └── cmu_mocap/            ← Upload dataset here
                ├── file1.bvh
                ├── file2.bvh
                └── ...
```

### Training Flow:

**Phase 1 (Now):**
1. Run Cell 5 → Trains for 12 hours
2. Colab disconnects (automatic)
3. Re-run Cell 5 → Resumes from `latest.pt` in Drive
4. Repeat 3-4 times until robot walks well

**Phase 2 (Later - after 3-4 Colab sessions):**
1. Upload CMU MoCap to Drive
2. Run Cell 6 → Check dataset
3. Run Cell 7 → Train on demonstrations
4. Get brain with natural movement!

### Benefits:
- ✅ **No manual downloads** - everything in Drive
- ✅ **Resume anytime** - checkpoint always saved
- ✅ **Access anywhere** - any device, any browser
- ✅ **Datasets persistent** - upload once, use forever

### Monitoring:
- Watch reward increase each epoch
- Early: -200 to -100
- Mid: -50 to 0
- Late: 0 to +100
- When reward > 0 consistently → robot can walk!

### Time Estimates:
- **T4 GPU (free):** ~12-18 hours total (3-4 sessions)
- **A100 GPU (Colab Pro):** ~4-6 hours (1 session)
- **Your CPU:** 8-9 days

**GPU is 10-15x faster!** 🚀